In [3]:
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import classification_report
import sklearn
import HTMLParser
import re
from nltk.stem.porter import PorterStemmer as porterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\shekh\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def cleanTweet(originalTweet):
    htmlParser = HTMLParser.HTMLParser()

    tweet = originalTweet
    # convert to lower case
    tweet = tweet.lower()
    #tweet = htmlParser.unescape(originalTweet);
    #tweet = tweet.decode('windows-1252').encode('ascii', 'ignore')
    #tweet = tweet.decode('windows-1252')
    #tweet = tweet.decode("utf8").encode('ascii', 'ignore')
    #tweet = re.sub(r'[^\x00-\xFF]+', r'', tweet)
    #tweet = re.sub(r'[^\x00-\x7F]+', r'', tweet)
    #tweet = tweet.decode('utf-8').strip()
    #tweet = tweet.decode('unicode_escape').encode('ascii','ignore')
    #tweet = tweet.encode('ascii','ignore')
    tweet = ''.join([i if ord(i) < 128 else ' ' for i in tweet])
    
    # remove URLs in tweet
    tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)

    # remove strings starting with @ in tweet
    tweet = re.sub(r'(\s)@\w+', r'', tweet)
    tweet = re.sub(r'@\w+', r'', tweet)
    
    # remove HTML tags from tweet
    tweet = re.sub('<[^<]+?>', '', tweet)

    # separates words joined with capital words.
    # E.g. DisplayIsAweson to Display Is Awesom
    #tweet = " ".join(re.findall('[A-Z][^A-Z]*', tweet));

    # remove exclamation and other unnecessary characters
    tweet = re.sub(r'[<>!#@$:.,%\?-]+', r'', tweet)

    
    
    tweet = re.sub(r' a ', r' ', tweet)
    tweet = re.sub(r' with ', r' ', tweet)
    tweet = re.sub(r' is ', r' ', tweet)
    tweet = re.sub(r' to ', r' ', tweet)
    tweet = re.sub(r' the ', r' ', tweet)
    tweet = re.sub(r' and ', r' ', tweet)
    tweet = re.sub(r' of ', r' ', tweet)
    tweet = re.sub(r' in ', r' ', tweet)
    tweet = re.sub(r' for ', r' ', tweet)
    tweet = re.sub(r' all ', r' ', tweet)
    tweet = re.sub(r' it ', r' ', tweet)
    tweet = re.sub(r' that ', r' ', tweet)
    tweet = re.sub(r' for ', r' ', tweet)
    tweet = re.sub(r' be ', ' ', tweet)
    tweet = re.sub(r' your ', r' ', tweet)
    tweet = re.sub(r' our ', r' ', tweet)
    tweet = re.sub(r' are ', r' ', tweet)
    tweet = re.sub(r' by ', r' ', tweet)
    tweet = re.sub(r' this ', r' ', tweet)
    tweet = re.sub(r' thi ', r' ', tweet)
    tweet = re.sub(r' from ', r' ', tweet)
    tweet = re.sub(r' or ', r' ', tweet)
    tweet = re.sub(r' website_redact ', r' ', tweet)
    tweet = re.sub(r' at ', r' ', tweet)
    tweet = re.sub(r' on ', r' ', tweet)
    tweet = re.sub(r' href= ', r' ', tweet)
    tweet = re.sub(r'\btarget=_blank\b', r' ', tweet)
    tweet = re.sub(r' also ', r' ', tweet)
    tweet = re.sub(r' can ', r' ', tweet)
    tweet = re.sub(r' call/text ', r' ', tweet)
    tweet = re.sub(r' email ', r' ', tweet)
    tweet = re.sub(r' viewingcontact ', r' ', tweet)
    tweet = re.sub(r' id ', r' ', tweet)
    tweet = re.sub(r' veri ', r' ', tweet)
    tweet = re.sub(r' kagglemanagercom ', r' ', tweet)
    tweet = re.sub(r' website_redact', r' ', tweet)
    # remove extra white spaces
    tweet = re.sub(r'\s+', r' ', tweet)
    
    # stemming
    stemmer = porterStemmer()
    stemmedTweet = [stemmer.stem(word) for word in tweet.split(" ")]
    stemmedTweet = " ".join(stemmedTweet)
    
    tweet = str(stemmedTweet)
    tweet = tweet.replace("'", "")
    
    return tweet

In [6]:
df = pd.read_json('C:/Users/shekh/Desktop/rentallisting/train.json')

In [7]:
df['interest_level'] = df['interest_level'].astype('category')
df['interest_level_codes'] = df['interest_level'].cat.codes

In [11]:
#featuresDF = df[['description', 'features']].apply(lambda x: ' '.join(x), axis=1)
description_column = df['description']
label_column = df['interest_level_codes']

In [14]:
description_col_list = description_column.values.tolist()
label_col_list = label_column.tolist()

In [15]:
# # clean all the tweets
# i = 0;
# cleanedDescriptionList = []
# for description in description_col_list:
#     #tweet.encode('utf-8').strip()
#     #tweet = tweet.decode("utf8").encode('ascii', 'ignore')
#     #print (i ,),
#     cleanedDescription = cleanTweet(description).encode('ascii', 'ignore').strip();
#     cleanedDescriptionList.append(cleanedDescription);
    
#     i += 1

In [9]:
#featureSet = df[['description','display_address']];

In [16]:
trainDescription, testDescription,  , testLabels = train_test_split(description_column, label_column, 
                                                                              test_size=0.25, random_state = 40);

In [17]:
# Create feature vectors
vectorizer = TfidfVectorizer(min_df=0.00125,
                             max_df = 0.80,
                             sublinear_tf=True,
                             use_idf=True,
                            stop_words=u'english',
                            analyzer='word',
                            ngram_range=(1,5),lowercase=True)

In [18]:
train_vectors = vectorizer.fit_transform(trainDescription);
test_vectors = vectorizer.fit_transform(testDescription);
#total_vectors = vectorizer.fit_transform(cleanedDescriptionList);

In [20]:
print train_vectors.shape;
print trainLabels.shape;


(2, 2)
(37014L,)


In [19]:
multinomialNBClassifier = MultinomialNB();
multinomialNBClassifier.fit(train_vectors, trainLabels)

ValueError: Found input variables with inconsistent numbers of samples: [2, 37014]

In [ ]:
predicted_labels = multinomialNBClassifier.predict(test_vectors)
predicted_class = predicted_labels.tolist()

In [ ]:
target_names = ['0', '1', '2']

In [ ]:
print(classification_report(testLabels, predicted_class, target_names=target_names))

In [ ]:
score = svmLinearKernelClassifier.score(X_test,actual_class);
print score;

In [ ]:
cvScores = cross_val_score(svmLinearKernelClassifier, feature_matrix_df, y_label, cv=10, scoring='f1_micro');
print cvScores;